In [40]:
import boto3
import pandas as pd
from io import StringIO
import io
import json
import warnings
import tempfile
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime
import sys
import os
from tempfile import TemporaryDirectory
import zipfile
import numpy as np
warnings.filterwarnings("ignore")
import pdfconv

In [18]:
print('Started')

Started


In [19]:
# Save the dictionary to a JSON file
file_path = 'matched_patients_dict.json'
with open(file_path, 'r') as json_file:
    patients = json.load(json_file)

print(f'Data saved to {file_path}')

Data saved to matched_patients_dict.json


In [20]:
print('Loaded')

Loaded


In [21]:
with open('conf.json' , 'r') as conf:
    config = json.loads(conf.read())
    dynamic_part = f"{config['client']}/{config['year']}/{config['month']}/{config['destination_configuration']['suffix']}"

if config is None:
    print('ERROR: Problem parsing config file')
    sys.exit()


In [22]:
def zip_pdfs_in_folder(folder_name, output_zip_name):
    # Check if the folder exists
    if not os.path.exists(folder_name):
        print(f"Folder '{folder_name}' not found.")
        return

    # List all PDF files in the folder
    pdf_files = [f for f in os.listdir(folder_name) if f.endswith(".pdf") or f.endswith(".xlsx")]

    # Check if there are any PDF files to zip
    if not pdf_files:
        print(f"No PDF files found in '{folder_name}'.")
        return

    # Create a ZIP file for compression
    with zipfile.ZipFile(output_zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for pdf_file in pdf_files:
            pdf_file_path = os.path.join(folder_name, pdf_file)
            zipf.write(pdf_file_path, os.path.basename(pdf_file))

    print(f"PDF files in '{folder_name}' have been zipped to '{output_zip_name}'.")


In [23]:

def get_boto3_session():
    try:
        aws_profile = 'paradocs'
        session = boto3.Session(profile_name=aws_profile)
    except Exception as e:
        print('Switching to Default AWS Profile...')
        session = boto3.Session()
    return session

In [24]:

def download_gaptracker(tmpdirname , config , s3_client):
    try:
        gaptracker_path  = f"{config['client']}/{config['year']}/{config['month']}/{config['source_configuration']['suffix']}/{config['gaptracker_filename']}"
        print('gaptracker_path', gaptracker_path)
        gaptracker_download_path = os.path.join(tmpdirname , config['gaptracker_filename'])
        s3_client.download_file(config['source_configuration']['bucket'] , gaptracker_path , gaptracker_download_path)
        return gaptracker_download_path
    except Exception as e:
        print(f"Error downloading {config['gaptracker_filename']} from S3: {str(e)}")

In [25]:

def replace_worksheet_with_dataframe(dataframe, gaptracker_download_path, new_open_gaps_sheetname):
    try:
        with pd.ExcelWriter(gaptracker_download_path, engine='openpyxl', mode='a' , if_sheet_exists='replace') as writer: 
            dataframe.to_excel(writer, sheet_name=new_open_gaps_sheetname,index=False)
#         s3 = boto3session.resource('s3')
#         dest_key = f'{destination_prefix}/{client}/{year}/{month}/{active_gap_tracker_filename}'
#         s3.Object(destination_bucket, dest_key).put(Body=open(tmp_file, 'rb'))
    except Exception as e:
        print(f"Could not update Assesment and Gap Tracker values\n\n", str(e))

In [88]:
def replace_sheet(new_open_gaps_df, patient_dict):

    # Extract the first row and save it for later use
    first_row = new_open_gaps_df.iloc[0]

    # Rename the columns to use the second row as column names
    new_open_gaps_df.columns = new_open_gaps_df.iloc[1]

    # Drop the first and second rows
    new_open_gaps_df = new_open_gaps_df.iloc[2:]
    actual_columns = new_open_gaps_df.columns
    count = 0

    new_open_gaps_rows = len(new_open_gaps_df)
    print('Total Rows in New Open Gaps: ', new_open_gaps_rows)
    
    fil = new_open_gaps_df['Appointment Date'].str.strip().str.len() == 0
    new_open_gaps_df.loc[fil, 'Appointment Date'] = np.NaN
    # new_open_gaps_df['Appointment Date'].fillna('NA')

    len(new_open_gaps_df['Appointment Date'])
    new_open_gaps_df['Appointment Date'].count()
    print('Lets verify', len(new_open_gaps_df[~new_open_gaps_df['Appointment Date'].isna()]))
#     patient_dict = { 'uniquemembergapid':'', 'Appointment_Date':''}

    print('total_matched_patients', len(patient_dict.keys()))


    for uniquemembergapid, inner_dict in patient_dict.items():

        condition = uniquemembergapid == new_open_gaps_df['uniquemembergapid']
        new_open_gaps_df.loc[condition, 'Appointment Date'] = inner_dict.get('appointment_date')
        new_open_gaps_df.loc[condition, 'Gap Tracker'] = 'Assessed and Present'
        new_open_gaps_df.loc[condition, 'Notes'] = 'Updated by pipeline'
        count+= 1


    
    new_open_gaps_df['DeployedDate'] = pd.to_datetime(new_open_gaps_df['DeployedDate'], errors='coerce')
    new_open_gaps_df['birthdate'] = pd.to_datetime(new_open_gaps_df['birthdate'], errors='coerce')
    new_open_gaps_df['analyticrundate'] = pd.to_datetime(new_open_gaps_df['analyticrundate'], errors='coerce')
    new_open_gaps_df['DeployedDate'] = new_open_gaps_df['DeployedDate'].dt.strftime('%#m/%#d/%y')
    new_open_gaps_df['birthdate'] = new_open_gaps_df['birthdate'].dt.strftime('%#m/%#d/%y')
    new_open_gaps_df['analyticrundate'] = new_open_gaps_df['analyticrundate'].dt.strftime('%#m/%#d/%y')
    
    
    new_open_gaps_rows_post_processing = len(new_open_gaps_df)
    print('Total Rows in New Open Gaps: ', new_open_gaps_rows_post_processing)
    print('Total Unique Filtered Patients (# of rows updated): ', count)
    print('Lets verify again', len(new_open_gaps_df[~new_open_gaps_df['Appointment Date'].isna()]))

    new_open_gaps_df = pd.concat([actual_columns.to_frame().T, new_open_gaps_df], ignore_index=True)
    new_open_gaps_df.columns = first_row
    return new_open_gaps_df


In [87]:
def get_unique_member_dict(ogt_filepath='ogt.xlsx'):
    new_open_gaps_df = pd.read_excel(ogt_filepath, 'New Open Gaps', skiprows=1)
    member_mappings = new_open_gaps_df[['uniquememberid', 'uniquemembergapid']].set_index('uniquemembergapid').to_dict()
    member_mappings = member_mappings.get('uniquememberid')
    return member_mappings

In [68]:
with tempfile.TemporaryDirectory() as tmpdirname:
    session = get_boto3_session()
    s3_client = session.client('s3')
    gaptracker_download_path = download_gaptracker(tmpdirname , config , s3_client)
    new_open_gaps_df = pd.read_excel(gaptracker_download_path, 'New Open Gaps', header=None)


    df = replace_sheet(new_open_gaps_df.copy(), patients)
    replace_worksheet_with_dataframe(df, gaptracker_download_path, 'New Open Gaps')
    memberid_dict = get_unique_member_dict(gaptracker_download_path)
    xmls_dict = {}
    for uniquemembergapid, inner_dict in patients.items():
        s3_xml_key = inner_dict.get('pdpin') + '_' +str(inner_dict.get('appointment_date'))+'.xml'
        uniqueprovidergroupid = inner_dict.get('uniqueprovidergroupid')
        new_filename = f"{memberid_dict.get(uniquemembergapid)}_{str(inner_dict.get('appointment_date'))}"
        xmls_dict.update({s3_xml_key : {'unique_providergroup_id':uniqueprovidergroupid, 'new_filename':new_filename}})

    for xml_filename, inner_dict in xmls_dict.items():
        source_bucket = config['source_configuration']['bucket']
        xmls_key  = f"{config['client']}/{config['year']}/{config['month']}/xmls/{xml_filename}"
        local_path = os.path.join(tmpdirname, os.path.basename(xml_filename))
        s3_client.download_file(source_bucket, xmls_key, local_path)
        print(local_path)        
        print('newfilename', inner_dict.get('new_filename'))
        print('unique_providergroup_id', inner_dict.get('unique_providergroup_id'))
        pdfconv.main(local_path, tmpdirname, inner_dict.get('new_filename'), inner_dict.get('unique_providergroup_id'))

    print('Started zipping...')
    output_zip_name = f"optum-submission-for-{config['client']}-{config['year']}-{config['month']}.zip"    # Replace with your desired output ZIP file name
    zip_pdfs_in_folder(tmpdirname, output_zip_name)
    print('Zipping Done!')
    print('Uplaoding!')
    s3 = get_boto3_session().resource('s3')
    s3.Object('gold-ccd-zone', f'{output_zip_name}').put(Body=open(output_zip_name, 'rb'))
    print('Success!!!')

gaptracker_path eventus-whole-health/2023/11/optum-gap-tracker/eData -OCTOBER Gap Tracker - Eventus WholeHealth 10.30.2023 .xlsx


KeyboardInterrupt: 